In [1]:
# As first_reg, but with features and labels

In [2]:
%run project_base.ipynb

In [3]:
# Set to False to use existing weights for the neural network.
# Set to True to continue training the weights from where we left off last time.
TRAIN = True

In [4]:
class BilinearInterp(FineGridPredictorMixin, BilinearInterpBase):
    pass
    
    
class NearestInterp(FineGridPredictorMixin, NearestInterpBase):
    pass


class PolyInterp(FineGridPredictorMixin, PolyInterpBase):
    pass

In [5]:
### DNN hyperparameters
hidden_units = [2000] * 10
# i.e. the number of fine grid points.
logits = (fine_grid_fineness.t + 1) * (fine_grid_fineness.x + 1)
drop_rate = 0.4
batch_norm = False
preprocessor = ScaleDataOverall()
uuid = None

### Training hyperparameters #1
gradient_clip = 5.0  # May be set to None to turn off gradient clipping

### Logging/saving hyperparameters
log_steps = 500  # How many steps to print the current loss.
model_dir = './saved_models/seventh_reg/'
model_dir = model_dir_str(model_dir, hidden_units, logits, drop_rate, 
                          preprocessor, batch_norm, uuid=uuid)

dnn = Sequential.create_dnn(hidden_units, logits, drop_rate=drop_rate,
                            model_dir=model_dir, log_steps=log_steps,
                            gradient_clip=gradient_clip, batch_norm=batch_norm)
preprocessor.load(model_dir)

INFO:tensorflow:Using config: {'_model_dir': './saved_models/seventh_reg/2000x10_121_D04_ScaleDataOverall', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 500, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f935b4b67b8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Processor checkpoint file found, restoring values.


In [ ]:
if TRAIN:
    ### Training hyperparameters #2
    batch_size = 30
    steps = 3000
    train_input_fn = preprocessor.init(BatchData(batch_size=batch_size))
    dnn.train(input_fn=train_input_fn(use_tf=True), max_steps=steps, 
              hooks=[ProcessorSavingHook(preprocessor, model_dir)])
    
    batch_size = 300
    steps = 6000
    train_input_fn = preprocessor.init(BatchData(batch_size=batch_size))
    dnn.train(input_fn=train_input_fn(use_tf=True), max_steps=steps, 
              hooks=[ProcessorSavingHook(preprocessor, model_dir)])

INFO:tensorflow:Skipping training since max_steps has already saved.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./saved_models/seventh_reg/2000x10_121_D04_ScaleDataOverall/model.ckpt-3481
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving processor checkpoint for 3481 into ./saved_models/seventh_reg/2000x10_121_D04_ScaleDataOverall
INFO:tensorflow:Saving checkpoints for 3482 into ./saved_models/seventh_reg/2000x10_121_D04_ScaleDataOverall/model.ckpt.
INFO:tensorflow:loss = 0.0045934333, step = 3482
INFO:tensorflow:Saving processor checkpoint for 3720 into ./saved_models/seventh_reg/2000x10_121_D04_ScaleDataOverall
INFO:tensorflow:Saving checkpoints for 3721 into ./saved_models/seventh_reg/2000x10_121_D04_ScaleDataOverall/model.ckpt.
INFO:tensorflow:Saving processor checkpoint for 3

In [ ]:
poly_deg = 10
results = plot_regressors([(Perfect(), None, 'Exact'),
                           (dnn, preprocessor.training(False), 'DNN'),
                           (BilinearInterp(), None, 'Bilinear'),
                           (PolyInterp(poly_deg), None, 'Poly-{}'.format(poly_deg))],
                          X_peak,
                          y_peak)

In [ ]:
results[1].loss, results[2].loss, results[3].loss